In [ ]:
from pathlib import Path
import os

DAY = "20251201"
Version = "v1"

PROJECT_ROOT = Path(os.getenv("LLMSC_ROOT", ".")).resolve()

DATA_DIR = Path(os.getenv("LLMSC_DATA_DIR", PROJECT_ROOT / "input")).resolve()
OUT_DIR  = Path(os.getenv("LLMSC_OUT_DIR",  PROJECT_ROOT / "runs" / f"{DAY}.{Version}")).resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
RANDOM_SEED = 42
import random, os
import numpy as np
random.seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"🔒 Random seed set to {RANDOM_SEED} for reproducibility.")

🔒 Random seed set to 42 for reproducibility.


https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE156728

In [ ]:
import os
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")
USE_LLM = bool(GEMINI_API_KEY)

if not USE_LLM:
    print("⚠️ GEMINI_API_KEY not set → LLM inference cells will be skipped.")

In [ ]:
import google.generativeai as genai
import scanpy as sc
import numpy as np
import pandas as pd
import scipy
import os
import gc
import time

import warnings
warnings.filterwarnings("ignore")


In [ ]:
# ==========================================
# 1. API
# ==========================================

genai.configure(api_key=GEMINI_API_KEY)
print("--- Available Models for your Key ---")
try:
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(m.name)
except Exception as e:
    print(f"Error: {e}")

--- Available Models for your Key ---
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-

In [ ]:
MODEL_NAME = 'models/gemini-2.5-pro'

# MODEL_NAME = 'models/gemini-3-pro-preview'
# MODEL_NAME = 'models/gemini-2.5-flash'
# MODEL_NAME = 'models/gemini-2.0-flash'

print(f"Using Model: {MODEL_NAME}")
model = genai.GenerativeModel(MODEL_NAME)

Using Model: models/gemini-2.5-pro


In [ ]:
!tar -xzvf $DATA_DIR/GSE176078_Wu_etal_2021_BRCA_scRNASeq.tar.gz

# 1. Wu_etal_2021_BRCA_scRNASeq/count_matrix_sparse.mtx
# 2. Wu_etal_2021_BRCA_scRNASeq/count_matrix_genes.tsv
# 3. Wu_etal_2021_BRCA_scRNASeq/count_matrix_barcodes.tsv
# 4. Wu_etal_2021_BRCA_scRNASeq/metadata.csv

In [ ]:
# ==========================================
# 2. Real Data Loading (Wu et al. 2021 BRCA)
# ==========================================
# Objective: Load Mesenchymal cells (CAFs/PVLs), filter based on provided metadata,
# and perform stratified subsampling to ensure balanced benchmarking.

print("🚀 Loading Wu et al. (2021) Breast Cancer Atlas (Real Data)...")

N_SAMPLES_PER_CLUSTER = 300
RANDOM_SEED = 42

try:
    # ---------------------------------------------------------
    # Step 1: Load Sparse Matrix and Features
    # ---------------------------------------------------------
    base_path = f"{DATA_DIR}/Wu_etal_2021_BRCA_scRNASeq"
    mtx_path = os.path.join(base_path, "count_matrix_sparse.mtx.gz")
    genes_path = os.path.join(base_path, "count_matrix_genes.tsv")
    barcodes_path = os.path.join(base_path, "count_matrix_barcodes.tsv")
    meta_path = os.path.join(base_path, "metadata.csv")

    print(f"  Reading Matrix: {mtx_path} ...")

    with gzip.open(mtx_path, "rb") as f:
      mat = io.mmread(f)

    print(f"  Reading Genes: {genes_path} ...")
    genes = pd.read_csv(genes_path, sep='\t', header=None)
    gene_names = genes[0].values

    print(f"  Reading Barcodes: {barcodes_path} ...")
    barcodes = pd.read_csv(barcodes_path, sep='\t', header=None)
    cell_names = barcodes[0].values

    print(f"  Raw Matrix shape: {mat.shape}, Genes: {len(gene_names)}, Cells: {len(cell_names)}")

    if mat.shape[0] == len(gene_names):
        print("  -> Transposing matrix to (Cells x Genes)...")
        mat = mat.tocsr().T
    elif mat.shape[0] == len(cell_names):
        print("  -> Matrix is already (Cells x Genes).")
        mat = mat.tocsr()
    else:
        raise ValueError("Matrix dimensions do not match gene or cell counts.")

    # ---------------------------------------------------------
    # Step 2: Create AnnData Object
    # ---------------------------------------------------------
    adata = sc.AnnData(mat)
    adata.obs_names = cell_names
    adata.var_names = gene_names
    adata.var_names_make_unique()

    del mat, genes, barcodes
    gc.collect()

    # ---------------------------------------------------------
    # Step 3: Integrate Metadata
    # ---------------------------------------------------------
    print(f"  Reading Metadata: {meta_path} ...")
    meta = pd.read_csv(meta_path, index_col=0)

    # Strict alignment of indices
    common_cells = adata.obs_names.intersection(meta.index)
    if len(common_cells) == 0:
        raise ValueError("No overlapping cells found between matrix and metadata.")

    print(f"  Merging metadata for {len(common_cells)} common cells...")
    adata = adata[common_cells]
    adata.obs = meta.loc[common_cells]

    # ---------------------------------------------------------
    # Step 4: Filter for Mesenchymal Lineage (CAFs/PVLs)
    # ---------------------------------------------------------
    target_subtypes = [
        'Endothelial ACKR1',
        'Endothelial RGS5',
        'Endothelial CXCL12',
        'CAFs MSC iCAF-like',
        'CAFs myCAF-like',
        'PVL Differentiated',
        'PVL Immature',
        'Cycling PVL'
    ]

    col_name = 'celltype_minor'

    if col_name not in adata.obs.columns:
         raise ValueError(f"Column '{col_name}' not found. Available keys: {adata.obs.columns.tolist()}")

    print(f"  Filtering for Mesenchymal subsets in '{col_name}'...")
    adata = adata[adata.obs[col_name].isin(target_subtypes)].copy()

    # Map to 'meta.cluster' for consistency with the pipeline
    adata.obs['meta.cluster'] = adata.obs[col_name]

    print(f"  -> Initial Mesenchymal Count: {adata.n_obs} cells")

    # ---------------------------------------------------------
    # Step 5: Stratified Subsampling
    # ---------------------------------------------------------
    # This ensures rare PVL subtypes are not overwhelmed by dominant CAF populations.

    print(f"  Performing Stratified Subsampling (Target: ~{N_SAMPLES_PER_CLUSTER} cells/cluster)...")

    subsampled_indices = []
    clusters = adata.obs['meta.cluster'].unique()

    for cls in clusters:
        cls_indices = adata[adata.obs['meta.cluster'] == cls].obs_names
        n_cells = len(cls_indices)

        if n_cells > N_SAMPLES_PER_CLUSTER:
            selected = np.random.RandomState(RANDOM_SEED).choice(
                cls_indices, N_SAMPLES_PER_CLUSTER, replace=False
            )
        else:
            selected = cls_indices # Keep all if below threshold

        subsampled_indices.extend(selected)
        print(f"    - {cls}: {n_cells} -> {len(selected)}")

    adata = adata[subsampled_indices].copy()

    # ---------------------------------------------------------
    # Step 6: Normalization & Preprocessing
    # ---------------------------------------------------------
    print("  Normalizing and Log-transforming...")
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.layers["logcounts"] = adata.X.copy()
    adata.raw = None
    adata.obs['Sample_ID'] = adata.obs.index

    save_path = OUT_DIR / "baca_msc_benchmark_data.h5ad"
    print(f"\n💾 Saving Benchmark Data to: {save_path}")
    adata.write_h5ad(save_path)

    meta_save_path =  OUT_DIR / "baca_msc_benchmark_metadata.csv"
    adata.obs.to_csv(meta_save_path)
    print(f"📄 Metadata saved to: {meta_save_path}")

    # ---------------------------------------------------------
    # Step 7: Final Validation
    # ---------------------------------------------------------
    print(f"✅ Loaded & Balanced BRCA MSC Data: {adata.n_obs} cells")
    print("   Final Cluster Distribution:")
    print(adata.obs['meta.cluster'].value_counts())

except Exception as e:
    print(f"❌ Critical Error loading real data: {e}")
    raise

🚀 Loading Wu et al. (2021) Breast Cancer Atlas (Real Data)...
  Reading Matrix: /input/Wu_etal_2021_BRCA_scRNASeq/count_matrix_sparse.mtx.gz ...
  Reading Genes: /input/Wu_etal_2021_BRCA_scRNASeq/count_matrix_genes.tsv ...
  Reading Barcodes: /input/Wu_etal_2021_BRCA_scRNASeq/count_matrix_barcodes.tsv ...
  Raw Matrix shape: (29733, 100064), Genes: 29733, Cells: 100064
  -> Transposing matrix to (Cells x Genes)...
  Reading Metadata: /input/Wu_etal_2021_BRCA_scRNASeq/metadata.csv ...
  Merging metadata for 100064 common cells...
  Filtering for Mesenchymal subsets in 'celltype_minor'...
  -> Initial Mesenchymal Count: 19398 cells
  Performing Stratified Subsampling (Target: ~300 cells/cluster)...
    - Endothelial ACKR1: 4611 -> 300
    - Endothelial RGS5: 1147 -> 300
    - Endothelial CXCL12: 1644 -> 300
    - CAFs MSC iCAF-like: 3153 -> 300
    - CAFs myCAF-like: 3420 -> 300
    - PVL Differentiated: 3487 -> 300
    - PVL Immature: 1886 -> 300
    - Cycling PVL: 50 -> 50
  Normalizin